In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 2.50MB/s]


In [2]:
!pip install kaggle

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m


 99% 1.43G/1.44G [00:16<00:00, 75.8MB/s]
100% 1.44G/1.44G [00:16<00:00, 91.7MB/s]


In [4]:
!7z e fasttext-crawl-300d-2m.zip -o/content -r



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1545551987 bytes (1474 MiB)

Extracting archive: fasttext-crawl-300d-2m.zip
--
Path = fasttext-crawl-300d-2m.zip
Type = zip
Physical Size = 1545551987

  0%      0% - crawl-300d-2M.vec                          1% - crawl-300d-2M.vec                          2% - crawl-300d-2M.vec                          3% - crawl-300d-2M.vec                          4% - crawl-300d-2M.vec                          5% - crawl-300d-2M.vec                        

In [5]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Reading glove vectors in python: https://stackoverflow.com/a/38230349/4084039
def fasttextModel(gloveFile):
    print ("Loading Fasttext Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}#for storing word and the corresponding embedding vector for that word
    for line in f:
        splitLine = line.split()#splitting the line and storing it in a list
        word = splitLine[0]#getting the first element and storing it in word
        embedding = np.array([float(val) for val in splitLine[1:]])#obtaining corresponding vector for that word
        model[word] = embedding#storing word as key and embedding vector for that word as value
    print ("Done.",len(model)," words loaded!")
    return model
model = fasttextModel('/content/crawl-300d-2M.vec')

Loading Fasttext Model
Done. 2000000  words loaded!


In [7]:
df=pd.read_csv('preprocessed_data.csv')#reading data into DataFrame

In [8]:
df.head(4)#displaying top 4 datapoints


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [9]:
def preprocess(x):#removing last character
  x=x[:-1]
  return x

In [10]:
df['source']=df['source'].apply(preprocess)#preprocessing source data
df['target']=df['target'].apply(preprocess)#preprocessing target data

In [11]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [12]:
df.shape#shape of DataFrame


(2000, 2)

In [13]:
df=df[df['source'].apply(len)<170]#removing source sentences of length greater than or equal to 170
df=df[df['target'].apply(len)<200]#removing target sentences of length greater than or equal to 200

In [14]:
df.shape#shape of DataFrame


(1990, 2)

In [15]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting the data in the ratio 99:1
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


Target:

In [16]:
target_tokenizer= Tokenizer(lower=False)#tokenization on target
target_tokenizer.fit_on_texts(y_train)#fitting on ytrain
target_vocab_size= len(target_tokenizer.word_index) + 1#target vocab size
print(len(target_tokenizer.word_index))

3600


In [17]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting text to integers
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting text to integers

In [18]:
target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding to maxlength

In [19]:
target_padded_docs_train.shape


(1970, 43)

In [20]:
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [21]:
target_padded_docs_test.shape


(20, 43)

Source:

In [22]:
source_tokenizer= Tokenizer(lower=False)#tokenization on source
source_tokenizer.fit_on_texts(X_train)#fitting to X_train
source_vocab_size= len(source_tokenizer.word_index) + 1#source vocab size
print(len(source_tokenizer.word_index))

4623


In [23]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting text to sequence
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting text to sequence

In [24]:
source_padded_docs_train = pad_sequences(source_encoded_docs_train,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [25]:
source_padded_docs_train.shape

(1970, 43)

In [26]:
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [27]:
source_padded_docs_test.shape

(20, 43)

In [28]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))

In [29]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [30]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional

source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))

In [31]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [32]:
#creating embedding matrix
embedding_matrix = np.zeros((source_vocab_size, 300))
for word, i in source_tokenizer.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Model:

In [33]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,300,weights=[embedding_matrix],input_length=source_padded_docs_train.shape[1],trainable=False)(input)
lstm1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embed)  
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(lstm1)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 43, 300)           1387200   
_________________________________________________________________
bidirectional (Bidirectional (None, 43, 256)           439296    
_________________________________________________________________
time_distributed (TimeDistri (None, 43, 3601)          925457    
Total params: 2,751,953
Trainable params: 1,364,753
Non-trainable params: 1,387,200
_________________________________________________________________


In [34]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [35]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 10s 907ms/step - loss: 8.0933 - accuracy: 0.3287 - val_loss: 6.4809 - val_accuracy: 0.7047
Epoch 2/50
2/2 [==============================] - 1s 260ms/step - loss: 5.7948 - accuracy: 0.6822 - val_loss: 3.0150 - val_accuracy: 0.6977
Epoch 3/50
2/2 [==============================] - 1s 265ms/step - loss: 3.2016 - accuracy: 0.6735 - val_loss: 2.9736 - val_accuracy: 0.6977
Epoch 4/50
2/2 [==============================] - 1s 261ms/step - loss: 3.2845 - accuracy: 0.6735 - val_loss: 2.5922 - val_accuracy: 0.6977
Epoch 5/50
2/2 [==============================] - 1s 263ms/step - loss: 2.8372 - accuracy: 0.6745 - val_loss: 2.3567 - val_accuracy: 0.7012
Epoch 6/50
2/2 [==============================] - 1s 264ms/step - loss: 2.6457 - accuracy: 0.6775 - val_loss: 2.3243 - val_accuracy: 0.7012
Epoch 7/50
2/2 [==============================] - 1s 266ms/step - loss: 2.5460 - accuracy: 0.6775 - val_loss: 2.1505 - val_accuracy: 0.7012
Epoch 8/50
2/2 [===

In [36]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=10,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/10
2/2 [==============================] - 1s 290ms/step - loss: 0.8384 - accuracy: 0.8356 - val_loss: 1.2536 - val_accuracy: 0.8047
Epoch 2/10
2/2 [==============================] - 1s 266ms/step - loss: 0.8204 - accuracy: 0.8374 - val_loss: 1.2511 - val_accuracy: 0.8023
Epoch 3/10
2/2 [==============================] - 1s 267ms/step - loss: 0.8040 - accuracy: 0.8393 - val_loss: 1.2533 - val_accuracy: 0.8058
Epoch 4/10
2/2 [==============================] - 1s 272ms/step - loss: 0.7863 - accuracy: 0.8412 - val_loss: 1.2699 - val_accuracy: 0.8047
Epoch 5/10
2/2 [==============================] - 1s 272ms/step - loss: 0.7721 - accuracy: 0.8430 - val_loss: 1.2544 - val_accuracy: 0.8058
Epoch 6/10
2/2 [==============================] - 1s 269ms/step - loss: 0.7573 - accuracy: 0.8446 - val_loss: 1.2635 - val_accuracy: 0.8023
Epoch 7/10
2/2 [==============================] - 1s 267ms/step - loss: 0.7431 - accuracy: 0.8470 - val_loss: 1.2635 - val_accuracy: 0.8047
Epoch 8/10
2/2 [====

In [50]:
from datetime import datetime
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
def function1(x):
  start = datetime.now()
  encoded=source_tokenizer.texts_to_sequences(x)
  padded=pad_sequences(encoded,maxlen=43,padding="post")
  padded=padded.reshape((*padded.shape,1))
  x=model.predict(padded)
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  print("The time for evaluation:"+str(datetime.now() - start))
  return ' '.join(y_lst)


In [51]:
x='wht r u doin?'
function1([x])

The time for evaluation:0:00:00.040164


'What are you doing'

In [52]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
def function2(x,y_true):
  start = datetime.now()
  encoded=source_tokenizer.texts_to_sequences(x)
  padded=pad_sequences(encoded,maxlen=43,padding="post")
  padded=padded.reshape((*padded.shape,1))
  x=model.predict(padded)
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  bleu_score=bleu.sentence_bleu([y_true.split(),],y_lst)
  print("The time for evaluation:"+str(datetime.now() - start))
  return bleu_score


In [53]:
x='wht r u doin?'
y='What are you doing?'

In [54]:
import nltk.translate.bleu_score as bleu
function2([x],y)

The time for evaluation:0:00:00.040790


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.7071067811865476

In [42]:
#https://github.com/bhattbhavesh91/tflite-tutorials/blob/master/tflite-part-2.ipynb
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [43]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

Before Quantization:

In [44]:
import os
model_name="final_model.h5"
model.save(model_name)
convert_bytes(get_file_size(model_name),"MB")

File size: 20.959 Megabytes


After Quantization:

In [45]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpqwp9h5wy/assets


INFO:tensorflow:Assets written to: /tmp/tmpqwp9h5wy/assets


In [46]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

2811504

In [47]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "MB")


File size: 2.681 Megabytes


Streamlit DemoVideo Link:-https://youtu.be/Tm5MaWqa7OA